# Showcase 2D KP

### Load Model & Dataset

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import src.real_bug_dataset
from tqdm.notebook import tqdm
from src.models.pose_resnet import BasicBlock, PoseResNet, Bottleneck
import src.bug_dataset
from src.real_bug_dataset import RealBugDataset
from src.eval.loss import JointsMSELoss
from src.eval.accuracies import accuracy
import torch
from torchvision import transforms
from torch.utils.data import DataLoader
%matplotlib widget
import matplotlib.pyplot as plt

In [ ]:
file_path = '../data/single_sungaya/label_names.txt'

with open(file_path) as file:
    label_names = file.readlines()
    label_names = [item.rstrip() for item in label_names]

In [ ]:
# Sungaya Synth
target_dir = "../data/single_sungaya/"
out_df = pd.read_hdf(os.path.join(target_dir, "Data_3D_Pose.hdf5"))
reduceKeypoints = True
sungaya_dataset_synth = src.bug_dataset.BugDataset(df=out_df,
                             root_dir=target_dir, reduced=reduceKeypoints, transform=transforms.Compose([
                                src.bug_dataset.ToTensor()
                                   ]))
                 

In [ ]:
# Sungaya Real
target_dir = "../data/real_sungaya/"
out_df = pd.read_csv(os.path.join(target_dir, "realdata.csv"))
sungaya_dataset = RealBugDataset(df=out_df, root_dir=target_dir, std=sungaya_dataset_synth.std_2d   , mean=sungaya_dataset_synth.means_2d  , transform=transforms.Compose([
                                src.real_bug_dataset.ToTensor()
                                   ]))


In [ ]:
batch_size = 128
test_dataloader_real = DataLoader(sungaya_dataset, batch_size=batch_size)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))   

# Create Model
num_layers = 18
resnet_spec = {18: (BasicBlock, [2, 2, 2, 2]),
               34: (BasicBlock, [3, 4, 6, 3]),
               50: (Bottleneck, [3, 4, 6, 3]),
               101: (Bottleneck, [3, 4, 23, 3]),
               152: (Bottleneck, [3, 8, 36, 3])}
if reduceKeypoints:
    block_class, layers = resnet_spec[num_layers]
    model = PoseResNet(block_class, layers, 28)

else:
    block_class, layers = resnet_spec[num_layers]
    model = PoseResNet(block_class, layers, 62)


criterion = JointsMSELoss(use_target_weight=True)
model = model.to(device)
path = "../models/2d_horiz_vertical_flip/simple_2d_model_20220321-195443_128"
model.load_state_dict(torch.load(path))
model.eval()
trained_model = model

In [ ]:
# trained_model = liftingtrainer.model
trained_model.eval()
# Code get the first batch of results
with torch.no_grad():
    for data in test_dataloader_real:
        image = data['image']
        target = data['heatmap']
        file_name = data['file_name']
        kp = data['key_points_2D']
        visib = data['visibility']
        input_var = image.to(device, dtype=torch.float)
        heatmap_var = target.to(device, dtype=torch.float)

        predi = trained_model(input_var)


In [ ]:
from src.eval.accuracies import get_max_preds
# Display a sample
sample = 11
limb = 3

name = file_name[sample]

img = image[sample].numpy()
img=np.swapaxes(img,0,1)
img=np.swapaxes(img,1,2)

pred = predi.cpu().numpy()

acc = target.cpu().numpy()
acc_kp = ((kp[sample].numpy()*sungaya_dataset.std_2d)+sungaya_dataset.means_2d).astype(int)

vis = visib[sample].cpu().numpy()
print("Limb Visible", vis[limb] , acc_kp[limb])
plt.figure(figsize=(8,6))
plt.imshow(img)

plt.scatter(acc_kp[limb,0], acc_kp[limb,1], marker='o', s=10,  color='b')

plt.imshow(cv2.resize(pred[sample,limb], (256,256)), alpha = .4,cmap='hot')

plt.colorbar()
plt.title(name)
plt.show()